# Preparação do ambiente

## Bibliotecas

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from itertools import pairwise
from pickle import load

## Constantes e funções auxiliares

In [2]:
FILE_SCH_DATABASE = '../datasets/sch/produtos_certificados.zip'
FILES_INSPECAO_ECOMMERCE = [
    '../datasets/inspecao_ecommerce/carrefour.parquet.gzip',   
    '../datasets/inspecao_ecommerce/ml.parquet.gzip', 
    '../datasets/inspecao_ecommerce/magalu.parquet.gzip']
FILE_MODEL = '../datasets/model/sgd_model_20240423.pkl'

In [3]:
def load_sch(sch_database_file):
    
    # load SCH database
    usecols = [0,1,11,12,13,14,15]
    dtype = {'Número de Homologação': 'str'}
    parse_dates = [0]
    date_format = '%d/%m/%Y'

    df_sch = pd.read_csv(
        sch_database_file,
        sep=';',
        usecols=usecols,
        dtype=dtype,
        parse_dates=parse_dates,
        date_format=date_format
        )

    df_sch = df_sch.sort_values(by='Data da Homologação',ascending=False)

    df_modelo = df_sch[['Número de Homologação','Modelo']].dropna()
    df_modelo = df_modelo.groupby('Número de Homologação',as_index=False)['Modelo'].apply(lambda x: ' | '.join(x))

    df_nome_comercial = df_sch[['Número de Homologação','Nome Comercial']].dropna()
    df_nome_comercial = df_nome_comercial.groupby('Número de Homologação',as_index=False)['Nome Comercial'].apply(lambda x: ' | '.join(x))

    columns_to_keep = ['Data da Homologação', 'Número de Homologação', 'Nome do Fabricante', 'Categoria do Produto', 'Tipo do Produto']
    df_sch = df_sch[columns_to_keep]
    df_sch = df_sch.drop_duplicates(subset='Número de Homologação')

    df_sch = df_sch.merge(df_modelo,how='left')
    df_sch = df_sch.merge(df_nome_comercial,how='left')
    df_sch = df_sch.fillna('')
    
    df_sch = df_sch.reset_index(drop=True)
    
    return df_sch 

In [4]:
def flatten_matrix(matrix):
    return [item for row in matrix for item in row]

In [5]:
def extract_amazon_images(row):
    imagens = []
    imagens.append(row['Imagem'].split()[0])
    imagens.extend(row['Imagens'].split())

    imagens = [imagem for imagem in imagens if imagem.startswith('http')]
    if len(imagens) >= 3:
        imagens = imagens[:3]
    elif len(imagens) == 2:
        imagens.extend([''])
    elif len(imagens) == 1:
        imagens.extend(['',''])
    else:
        imagens.extend(['','',''])

    return pd.Series(imagens)

def extract_amazon_desc(row):
    desc = '\n'.join(row)
    desc = desc.replace(NULL_STRING,' ')
    desc = re.sub('\s+', ' ', desc)
    return desc

In [6]:
def extract_images(row):
    imagens = []
    imagens.append(row['Imagem'].split()[0])
    
    if not pd.isna(row['Imagens']):
        imagens.extend(row['Imagens'].split(','))

    imagens = [imagem for imagem in imagens if imagem.startswith('http')]
    if len(imagens) >= 3:
        imagens = imagens[:3]
    elif len(imagens) == 2:
        imagens.extend([''])
    elif len(imagens) == 1:
        imagens.extend(['',''])
    else:
        imagens.extend(['','',''])

    return pd.Series(imagens)

In [7]:
def format_issue(row,remove_punct=False):
    nome,desc,img0,img1,img2 = row
    
    # remove null string
    desc = desc.replace(NULL_STRING,'')
    # remove punctiation from nome and desc
    if remove_punct:
        nome = nome.translate(str.maketrans('', '', string.punctuation))
        desc = desc.translate(str.maketrans('', '', string.punctuation))

    # remove ponto-e-vírgula e aspas duplas para evitar a 
    # identificação incorreta de campos ao importar para o redmine
    desc = desc.replace(';',',')
    desc = desc.replace('"',"''")
    # remove multiple linebrakes and spaces
    desc = re.sub('\n+','\n\n',desc)
    desc = re.sub('\s+',' ',desc)
    desc = desc.strip()
    
    img_css = '{max-width:250px; height:auto}'
    if len(img2) > 0:
        img_list = f'!{img_css}{img0}! !{img_css}{img1}! !{img_css}{img2}!'
    elif len(img1) > 0:
        img_list = f'!{img_css}{img0}! !{img_css}{img1}!'
    elif len(img0) > 0:
        img_list = f'!{img_css}{img0}!'
    else:
        img_list = ''
        
    if len(desc) >= 1:
        issue = f'h1. {nome}\n\n{desc}\n\n{img_list}'
    else:
        issue = f'h1. {nome}\n\n{img_list}'
    
    return issue.strip()

In [8]:
def plot_donut(labels,values,title,startangle=30):
    fig, ax = plt.subplots(figsize=(5, 3), subplot_kw=dict(aspect="equal"))
    
    wedges, texts = ax.pie(values, wedgeprops=dict(width=0.5), startangle=startangle)
    
    bbox_props = dict(boxstyle="round,pad=0.3", fc="w", ec="k", lw=0.72)
    kw = dict(arrowprops=dict(arrowstyle="-"),
              bbox=bbox_props, zorder=0, va="center")
    
    kw = dict(arrowprops=dict(arrowstyle="-"),bbox=bbox_props,
              zorder=0, va="center")
    
    for i, p in enumerate(wedges):
        ang = (p.theta2 - p.theta1)/2. + p.theta1
        y = np.sin(np.deg2rad(ang))
        x = np.cos(np.deg2rad(ang))
        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = "angle,angleA=0,angleB={}".format(ang)
        kw["arrowprops"].update({"connectionstyle": connectionstyle})
        ax.annotate(labels[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                    horizontalalignment=horizontalalignment, **kw)
    
    ax.set_title(title)
    plt.show()

# Carga e prepação dos dados

## Base SCH

In [9]:
df_sch = load_sch(FILE_SCH_DATABASE)
df_sch.head()

,Data da Homologação,Número de Homologação,Nome do Fabricante,Categoria do Produto,Tipo do Produto,Modelo,Nome Comercial
0,2024-07-01,072302416970,"Shenzhen Feasycom Co., Ltd.",2,Transceptor de Radiação Restrita,FSC-BT836B,
1,2024-07-01,185742113346,"Dongguan Aierpu Electronic Tecnology CO.,LTD",2,Transceptor de Radiação Restrita,XDG-79 | HSD-1511BT | XDG-138 | KMS-6686 | XDG...,
2,2024-07-01,013762200160,Intelbras S.a. Industria de Telecomunicacao El...,1,ONT - Terminação de Rede Ótica,WiFiber 1200R | WiFiber 1200R,
3,2024-07-01,064212416867,"Desonic (Huizhou) Electronics Co., Ltd.",2,Transceptor de Radiação Restrita,DS-582-SP,
4,2024-07-01,050922416680,"SICHUAN TIANYI COMHEART TELECOM CO., LTD",1,ONT - Terminação de Rede Ótica,TG6241 | TG6241,


## Anúncios coletados

In [10]:
df_inspecao_ecommerce = pd.concat([pd.read_parquet(file) for file in FILES_INSPECAO_ECOMMERCE])
df_inspecao_ecommerce = df_inspecao_ecommerce.reset_index(drop=True)
df_inspecao_ecommerce['Descrição'] = df_inspecao_ecommerce['Descrição'].fillna('')
df_inspecao_ecommerce

,Nome,Preço_Original,Preço,Imagem,Link,Data_Atualização,Palavra_Chave,Categoria,Marca,Vendedor,...,EAN,Certificado_de_Homologação,Características,Link_Pagina,Acessado,Qtd_vendidos,Qtd_disponível,Estado,Avaliações,Nota
0,Controle Sem Fio De Xbox Carbon Black Wireless...,<NA>,"R$ 470,69",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/controle-sem-fio-...,2024-06-19T14:30:19,bluetooth,Consoles e Games|Acessórios gamer|Joystick e C...,<NA>,,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,Headset Gamer Logitech G435 Sem Fio Bluetooth ...,<NA>,"R$ 449,90",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/headset-gamer-log...,2024-06-19T14:30:24,bluetooth,Informática|Acessórios para Informática|Fone d...,<NA>,,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Fone de Ouvido Headset Tectoy Bluetooth XPEAKE...,"R$ 92,60","R$ 49,98",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/fone-de-ouvido-he...,2024-06-19T14:30:32,bluetooth,Informática|Acessórios para Informática|Fone d...,<NA>,,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,Capa Teclado Bluetooth Anti Impacto Ipad 9 10....,<NA>,"R$ 569,00",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/capa-teclado-blue...,2024-06-19T14:30:39,bluetooth,"Celulares, Smartphones e Smartwatches|Acessóri...",<NA>,,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,Multimídia Automotivo Pioneer Dmh-a248bt Webli...,<NA>,"R$ 1.197,69",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/multimidia-automo...,2024-06-19T14:30:46,bluetooth,Automotivo|Som Automotivo|Rádio Automotivo|Mul...,<NA>,,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7142,Smart Tv Lg 32 Hd 32Lq620 Wi-Fi Bluetooth Hdr ...,1656.23,1490.61,https://a-static.mlcdn.com.br/280x210/smart-tv...,https://www.magazineluiza.com.br/smart-tv-lg-3...,2024-06-18 11:44:54,bluetooth-wifi,TV e Vídeo | TVs,LG,Olist Plus,...,<NA>,80281811799.0,"{'Apps integradas': 'LG Content Store', 'Quant...",<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>
7143,"Smart TV LG 43'' 4K Ultra HD 43UR7800PSA, WiFi...",<NA>,2118.1,https://a-static.mlcdn.com.br/280x210/smart-tv...,https://www.magazineluiza.com.br/smart-tv-lg-4...,2024-06-18 11:44:53,bluetooth-wifi,TV e Vídeo | TVs | TV 4K / UHD,<NA>,Miranda Computação,...,<NA>,<NA>,"{'NOME': '188302201925', 'Informações compleme...",<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>
7144,Carregador de Parede USB 2.1A 10W Bivolt - Bra...,43.79,40.29,https://a-static.mlcdn.com.br/280x210/carregad...,https://www.magazineluiza.com.br/carregador-de...,2024-06-18 11:44:47,carregador-celular,Celulares e Smartphones | Acessórios para Celu...,<NA>,Loja Ibyte,...,<NA>,<NA>,"{'marca': 'Goldentec', 'cor': 'Branco', 'entra...",<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>
7145,Carregador De Parede Usams 20W Pd Com Porta Ti...,45.7,41.13,https://a-static.mlcdn.com.br/280x210/carregad...,https://www.magazineluiza.com.br/carregador-de...,2024-06-18 11:44:44,carregador-celular,Celulares e Smartphones | Acessórios para Celu...,Usams,Loja Olist,...,<NA>,<NA>,{'Modelo do carregador': 'Carregador de parede...,<NA>,False,<NA>,<NA>,<NA>,<NA>,<NA>


## Modelo pré-treinado

In [11]:
with open(FILE_MODEL, 'rb') as f:
    clf = load(f)
clf

Pipeline(steps=[('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 stop_words=['a', 'à', 'ao', 'aos', 'aquela',
                                             'aquelas', 'aquele', 'aqueles',
                                             'aquilo', 'as', 'às', 'até', 'com',
                                             'como', 'da', 'das', 'de', 'dela',
                                             'delas', 'dele', 'deles', 'depois',
                                             'do', 'dos', 'e', 'é', 'ela',
                                             'elas', 'ele', 'eles', ...])),
                ('transformer', TfidfTransformer()),
                ('clf', SGDClassifier(alpha=1e-05, loss='log_loss'))])

# Análise

In [12]:
# df.iloc[5013]
# df_sch[df_sch['Número de Homologação']=='076211902585']

In [13]:
docs = df_inspecao_ecommerce['Nome'].values
passivel = clf.predict(docs)
proba = clf.predict_proba(docs)[:,1]

map_passivel = {0: 'Não', 1: 'Sim'}

df_inspecao_ecommerce['Passível'] = passivel
df_inspecao_ecommerce['Passível'] = df_inspecao_ecommerce['Passível'].map(map_passivel)
df_inspecao_ecommerce['Probabilidade'] = proba

df_inspecao_ecommerce.head()

,Nome,Preço_Original,Preço,Imagem,Link,Data_Atualização,Palavra_Chave,Categoria,Marca,Vendedor,...,Características,Link_Pagina,Acessado,Qtd_vendidos,Qtd_disponível,Estado,Avaliações,Nota,Passível,Probabilidade
0,Controle Sem Fio De Xbox Carbon Black Wireless...,<NA>,"R$ 470,69",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/controle-sem-fio-...,2024-06-19T14:30:19,bluetooth,Consoles e Games|Acessórios gamer|Joystick e C...,<NA>,,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Sim,0.998699
1,Headset Gamer Logitech G435 Sem Fio Bluetooth ...,<NA>,"R$ 449,90",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/headset-gamer-log...,2024-06-19T14:30:24,bluetooth,Informática|Acessórios para Informática|Fone d...,<NA>,,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Sim,0.976602
2,Fone de Ouvido Headset Tectoy Bluetooth XPEAKE...,"R$ 92,60","R$ 49,98",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/fone-de-ouvido-he...,2024-06-19T14:30:32,bluetooth,Informática|Acessórios para Informática|Fone d...,<NA>,,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Sim,0.997419
3,Capa Teclado Bluetooth Anti Impacto Ipad 9 10....,<NA>,"R$ 569,00",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/capa-teclado-blue...,2024-06-19T14:30:39,bluetooth,"Celulares, Smartphones e Smartwatches|Acessóri...",<NA>,,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Sim,0.958329
4,Multimídia Automotivo Pioneer Dmh-a248bt Webli...,<NA>,"R$ 1.197,69",https://carrefourbr.vtexassets.com/arquivos/id...,https://www.carrefour.com.br/multimidia-automo...,2024-06-19T14:30:46,bluetooth,Automotivo|Som Automotivo|Rádio Automotivo|Mul...,<NA>,,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Sim,0.993051


In [14]:
df_resumo = df_inspecao_ecommerce.pivot_table(index='Palavra_Chave',columns='Passível', values='Nome', aggfunc='count').fillna(0)
df_resumo = df_resumo.astype('int')
df_resumo['Total'] = df_resumo.sum(axis=1)
df_resumo.loc['Total'] = df_resumo.sum()
df_resumo


Passível,Não,Sim,Total
Palavra_Chave,,,
bateria-celular,22,226,248
bloqueador-portatil,39,4,43
bloqueador-sinal,55,2,57
bluetooth,14,667,681
bluetooth-wifi,0,163,163
carregador celular,79,458,537
carregador-celular,17,427,444
carregador-para-smartphone,34,84,118
celulares-e-smartphones,5,369,374
